# Brecha Salarial de Género en Argentina

Este proyecto fue realizado por el grupo 15 constituido por Daniela Ferraro y Jorgelina Sigal. Se utilizó un conjunto de datos provenientes del Ministerio de Trabajo, Empleo y Seguridad Social que contiene información sobre las remuneraciones en Argentina, desagregadas por género, tipo de industria y fecha. El objetivo es analizar las brechas salariales de género y su evolución. 

### Dataset
https://www.trabajo.gob.ar/estadisticas/oede/empleoyremuneracion.asp

### Contenido del dataset
#### Periodo: 
Indica año y mes al que corresponde la remuneración.
#### Industria: 
Nombre del sector o rama de actividad económica.
#### Género: 
Género del grupo de trabajadores al que pertenece la remuneración.
#### Mujer: 
Variable binaria (1 o 0) toma el valor 1 si es mujer, 0 si no. Útil para análisis estadístico, modelos y gráficos.
#### Varon: 
Variable binaria (1 o 0) toma el valor 1 si es varón, 0 si no. Útil para análisis estadístico, modelos y gráficos.
#### Remuneración: 
Remuneración promedio mensual para ese sector, género y período (en pesos argentinos).


### Objetivo general
Analizar la evolución de la remuneración media del empleo registrado en el sector privado en Argentina, según rama de actividad y sexo, en el período comprendido entre enero de 1996 y octubre de 2022, utilizando herramientas de data science, con el fin de identificar patrones, desigualdades y tendencias vinculadas a la brecha salarial de género en el mundo laboral.

### Objetivos específicos
- Procesar y limpiar el conjunto de datos proporcionado mediante herramientas de data science, utilizando bibliotecas como pandas para su análisis exploratorio.

- Describir la evolución temporal de la remuneración media del empleo registrado en el sector privado, diferenciando por sexo y rama de actividad (según el CIIU rev. 3 a dos dígitos).

- Comparar las trayectorias salariales entre varones y mujeres en los distintos sectores de actividad, identificando las ramas con mayores y menores brechas de género.

- Visualizar los resultados mediante gráficos que faciliten la comprensión de las desigualdades salariales en el período analizado.

- Reflexionar sobre las implicancias sociales y económicas de las desigualdades de género en el mundo laboral formal, a partir de los hallazgos del análisis de datos.

### Tecnologías
- Python 3
- Pandas
- Jupyter Notebook
- Visual Studio Code
- Matplotlib 
- Seaborn

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

## Exploración de los datos

In [14]:
remuneraciones1 = pd.read_csv('Remuneraciones (1).csv', header=None)

In [7]:
# Muestra las primeras 5 filas
print(remuneraciones1.head())

  Período (mes año) agricultura y ganaderia agricultura y ganaderia.1  \
0               NaN                 Mujeres                   Varones   
1            ene-96                   380.2                     399.8   
2            feb-96                   406.8                     398.1   
3            mar-96                   399.7                     393.3   
4            abr-96                   375.9                     394.6   

  Silvicultura, extracción de madera Silvicultura, extracción de madera.1  \
0                            Mujeres                              Varones   
1                              426.2                                335.1   
2                                416                                338.7   
3                              416.5                                324.2   
4                              396.7                                334.1   

  Pesca y actividades relacionadas con la pesca  \
0                                       Mujeres

In [8]:
# Devuelve un objeto Index con los nombres
print(remuneraciones1.columns)

Index(['Período (mes año)', 'agricultura y ganaderia',
       'agricultura y ganaderia.1', 'Silvicultura, extracción de madera',
       'Silvicultura, extracción de madera.1',
       'Pesca y actividades relacionadas con la pesca',
       'Pesca y actividades relacionadas con la pesca.1',
       'Extracción de petróleo crudo y gas natural',
       'Extracción de petróleo crudo y gas natural.1',
       'Extracción de minerales metalíferos',
       ...
       'Eliminación de desperdicios', 'Eliminación de desperdicios.1',
       'Servicios de organizaciones empresariales',
       'Servicios de organizaciones empresariales.1',
       'Cinematografía, radio y televisión',
       'Cinematografía, radio y televisión.1', 'Servicios n.c.p.',
       'Servicios n.c.p..1', 'sin definir', 'sin definir.1'],
      dtype='object', length=115)


In [9]:
# Muestra el tipo de dato de cada columna
print(remuneraciones1.dtypes)

Período (mes año)                       object
agricultura y ganaderia                 object
agricultura y ganaderia.1               object
Silvicultura, extracción de madera      object
Silvicultura, extracción de madera.1    object
                                         ...  
Cinematografía, radio y televisión.1    object
Servicios n.c.p.                        object
Servicios n.c.p..1                      object
sin definir                             object
sin definir.1                           object
Length: 115, dtype: object


In [10]:
# Incluye nombre de columnas, tipos de datos y valores no nulos
remuneraciones1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 323 entries, 0 to 322
Columns: 115 entries, Período (mes año) to sin definir.1
dtypes: object(115)
memory usage: 290.3+ KB


In [11]:
# Estadisticas descriptivas 
print(remuneraciones1.describe())

       Período (mes año) agricultura y ganaderia agricultura y ganaderia.1  \
count                322                     323                       323   
unique               322                     318                       317   
top               oct-22                     405                    1342.2   
freq                   1                       3                         2   

       Silvicultura, extracción de madera  \
count                                 323   
unique                                320   
top                                 398.1   
freq                                    2   

       Silvicultura, extracción de madera.1  \
count                                   323   
unique                                  319   
top                                   310.9   
freq                                      2   

       Pesca y actividades relacionadas con la pesca  \
count                                            323   
unique                             

In [12]:
# Valores nulos o faltantes
print(remuneraciones1.isnull().sum())

Período (mes año)                       1
agricultura y ganaderia                 0
agricultura y ganaderia.1               0
Silvicultura, extracción de madera      0
Silvicultura, extracción de madera.1    0
                                       ..
Cinematografía, radio y televisión.1    0
Servicios n.c.p.                        0
Servicios n.c.p..1                      0
sin definir                             0
sin definir.1                           0
Length: 115, dtype: int64


## Transformación del dataset de remuneraciones

1. Se parte de un dataset reducido para facilitar el análisis y tener un objetivo claro (hoja tanto del dataset original).
2. Se convierte el dataset de formato horizontal a vertical.
3. Se extraen encabezados que estaban en múltiples filas.
4. Se ajustan nombres de columnas y se separan variables combinadas.
5. Se guarda el resultado limpio en un nuevo archivo CSV.


In [20]:
# Extracción de encabezados
industria_header = remuneraciones1.iloc[0]
genero_header = remuneraciones1.iloc[1]

# Eliminación de las primeras filas y reset de índice
remuneracion_data = remuneraciones1.iloc[2:].reset_index(drop=True)

# Agrupación de columnas como tuplas
remuneracion_data.columns = pd.MultiIndex.from_arrays([industria_header, genero_header])

# Transformación de columnas multiindex a texto simple
remuneracion_data.columns = [
    "Periodo" if i == 0 else f"{col[0]}_{col[1]}" 
    for i, col in enumerate(remuneracion_data.columns)
]

# Transformación de horizontal a vertical
remuneraciones1 = remuneracion_data.melt(id_vars='Periodo', var_name='Industria_Genero', value_name='Valor')

# Separación de columna combinada
remuneraciones1[['Industria', 'Genero']] = remuneraciones1['Industria_Genero'].str.rsplit('_', n=1, expand=True)

# Reordenación de las columnas
remuneraciones = remuneraciones[['Periodo', 'Industria', 'Genero', 'Valor']]

# Opcional: Asegurarse de que 'Valor' esté en formato numérico (por si es necesario hacer cálculos)
remuneraciones['Valor'] = pd.to_numeric(remuneraciones['Valor'], errors='coerce')

# Creación de una nueva columna combinada 'Industria_Genero' para codificar
remuneraciones['Industria_Genero'] = remuneraciones['Industria'] + '_' + remuneraciones['Genero']

# Aplicación de OneHotEncoder sobre la columna combinada
encoder = OneHotEncoder(sparse_output=False)
encoded = encoder.fit_transform(remuneraciones[['Industria_Genero']])

# Creación de DataFrame con las columnas codificadas
encoded_columns = encoder.categories_[0]
encoded_df = pd.DataFrame(encoded, columns=[f'Industria_Genero_{col}' for col in encoded_columns])

# Concatenación de los resultados con el DataFrame original
remuneraciones = pd.concat([remuneraciones, encoded_df], axis=1)

# Eliminación de las columnas originales 'Industria', 'Genero' y la columna combinada
remuneraciones = remuneraciones.drop(columns=['Industria', 'Genero', 'Industria_Genero'])

# Creación de nuevo archivo CSV con los datos procesados
remuneraciones.to_csv("Rem2.csv", index=False)

# Visualización del resultado final
print(remuneraciones.head())


  Periodo  Valor  Industria_Genero_Actividades de informática_Mujeres  \
0  ene-96  380.2                                                0.0     
1  feb-96  406.8                                                0.0     
2  mar-96  399.7                                                0.0     
3  abr-96  375.9                                                0.0     
4  may-96  395.6                                                0.0     

   Industria_Genero_Actividades de informática_Varones  \
0                                                0.0     
1                                                0.0     
2                                                0.0     
3                                                0.0     
4                                                0.0     

   Industria_Genero_Agencias de empleo temporario_Mujeres  \
0                                                0.0        
1                                                0.0        
2                            

In [13]:
# Ver cuántos valores únicos hay en 'Industria'
print(remuneraciones1['Industria'].nunique())

KeyError: 'Industria'

In [ ]:
# Ver todos los valores únicos en 'Industria'
print(remuneraciones1['Industria'].unique())

['agricultura y ganaderia' 'Silvicultura, extracción de madera'
 'Pesca y actividades relacionadas con la pesca'
 'Extracción de petróleo crudo y gas natural'
 'Extracción de minerales metalíferos'
 'Explotación de otras minas y canteras' 'Alimentos' 'Tabaco'
 'Productos textiles' 'Confecciones' 'Cuero' 'Madera' 'Papel' 'Edición'
 'Productos de petróleo' 'Productos químicos'
 'Productos de caucho y plástico' 'Otros minerales no metálicos'
 'Metales comunes' 'Otros productos de metal' 'Maquinaria y equipo'
 'Maquinaria de oficina' 'Aparatos eléctricos' 'Radio y televisión'
 'Instrumentos médicos' 'Automotores' 'Otros equipo de transporte'
 'Muebles' 'Reciclamiento de desperdicios y desechos'
 'Electricidad, gas y agua' 'Captación, depuración y distribución de agua'
 'Construcción'
 'Vta. y reparación de vehículos, vta. por menor de combustible'
 'Comercio al por mayor' 'Comercio al por menor'
 'Servicios de hotelería y restaurantes'
 'Transporte ferroviario y automotor y por tuberías'
 

### Mediante el analisis exploratorio de datos, el diagnostico de las columnas arrojo que el dataframe presenta las siguientes vaiables:
- Periodo: cuantitativa discreta. Unidades de tiempo.
- Industria: categorica/ cualitativa nominal. Representa categorias sin orden.
- Genero: categorica/ cualitativa nominal. Posee dos valores posibles.
- Valor: cuantitativa continua. Representa monto monetario.
### En base a estos resultados se avanza con la transformacion

In [12]:
# Se quitan puntos de miles
remuneraciones['Valor'] = remuneraciones['Valor'].str.replace('.', '', regex=False)
# Se reemplaza coma decimal por punto
remuneraciones['Valor'] = remuneraciones['Valor'].str.replace(',', '.', regex=False)
# Se convierte de object a float
remuneraciones['Valor'] = pd.to_numeric(remuneraciones['Valor'], errors='coerce')

In [13]:
# Para usar luego OneHotEncoder
remuneraciones['Genero'] = remuneraciones['Genero'].astype('category')
remuneraciones['Industria'] = remuneraciones['Industria'].astype('category')